# 5. Calculate Sheet fluxes Module

# 5.1 Import Functions

In [ ]:
import os
import glob
import datetime
import pandas as pd
import numpy as np
import xarray as xr
#import WAPORWA modules
os.chdir(r'C:\WA_Souss_Massa_Training\WAPOR\modules') #change to modules path
import WA
from WA.pickle_basin import pickle_in,pickle_out  
from WA.model_SMBalance import open_nc
from WA import GIS_functions as gis
from dask.distributed import Client

#Read pickle
Main_dir=r"C:\WA_Souss_Massa_Training\WAPOR\Souss_Massa\Main"
pickle=glob.glob(os.path.join(Main_dir,'*.pickle'))[-1] 
BASIN=pickle_in(pickle)  

#Customize dask performance
client = Client()
client

In [ ]:
time='A-{0}'.format(BASIN['end_month'])
for key in ['p','et','etincr','etrain']:
    nc=BASIN['main_data']['monthly'][key]
    var,name=open_nc(nc)
    var_y=var.resample(time=time).sum(dim='time',skipna=False)
    outfolder=os.path.join(BASIN['Dir'],'data','nc')  
    attrs={"units":"mm/year", "source": "Hydrolological year ", "quantity":name}
    var_y.assign_attrs(attrs)
    var_y.name = name
    var_y_dts=var_y.chunk({"latitude":-1,"longitude":-1}).to_dataset()
    nc_fn="{0}_hyearly.nc".format(key)
    nc_path=os.path.join(outfolder,nc_fn)
    var_y_dts.to_netcdf(nc_path,encoding={name:{'zlib':True}})
    BASIN['main_data']['yearly'][key]=nc_path
pickle_out(BASIN)

In [ ]:
#%% Create Area mask
# get template from LU map
template=glob.glob(os.path.join(BASIN['input_data']['yearly']['lu'][0],"*.tif"))[0]
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(template)
# calculate area per pixel
area_map=gis.MapPixelAreakm(template)
# save area map as tif

Dir_stat = os.path.join(BASIN['Dir'],'data','stat')
if not os.path.exists(Dir_stat):
    os.makedirs(Dir_stat)

BASIN['input_data']['stat']['area']=os.path.join(Dir_stat,'Area_km.tif')    
gis.CreateGeoTiff(BASIN['input_data']['stat']['area'],area_map,driver, NDV, xsize, ysize, GeoT, Projection)

# create area mask
shape=BASIN['geo_data']['basin'] #Shapefile of the area of interest
area=BASIN['input_data']['stat']['area']
BASIN['main_data']['stat']['area']=os.path.join(BASIN['Dir'],'data','stat','Basin_Area_km.tif')
gis.Clip_shapefile(area,shape,BASIN['main_data']['stat']['area'])

# 5.2 Calculate Fluxes

In [ ]:
### Select area of interest
area_fh=BASIN['main_data']['stat']['area'] # The whole Basin area
# area_fh=Basin['main_data']['subbasin']['Zarqa'] #or sub-basin area
area=gis.OpenAsArray(area_fh,nan_values=True)

### Calculate total P, ET, ETincr, ETrain
ts_all=[] #all timeseries
for key in ['p','et','etrain','etincr']:
    nc=BASIN['main_data']['yearly'][key]
    var,name=open_nc(nc)
    Volume=var*area
    attrs=var.attrs
    attrs['units']='TCM/year'
    Volume.assign_attrs(attrs)
    ts=Volume.sum(dim=['latitude','longitude']).to_dataframe()
    ts.index=[y.year for y in ts.index]
    ts_all.append(ts)
    
### Calculate ET consumptions by Land Use Categories
LU,_=open_nc(BASIN['main_data']['yearly']['lu'])
ETg,_=open_nc(BASIN['main_data']['yearly']['etrain'])
ETb,_=open_nc(BASIN['main_data']['yearly']['etincr'])


### Different year date to year 
LU=LU.groupby('time.year').first(skipna=False)
ETb=area*ETb.groupby('time.year').first(skipna=False)
ETg=area*ETg.groupby('time.year').first(skipna=False)
### average per LU
from WA.average_by_LU import Total_perLU
ts_ETb=Total_perLU(ETb,LU)
ts_ETg=Total_perLU(ETg,LU)

ts_all.append(ts_ETb)
ts_all.append(ts_ETg)

In [ ]:
### Read yearly dS from GRACE
df_dS_y=pd.read_csv(BASIN['input_ts']['dS_yearly'][0],sep=';',index_col=0)
Area_skm=np.nansum(area)
df_grace_ds=df_dS_y*Area_skm
df_grace_ds.index=[int(y[0:4]) for y in df_grace_ds.index]
df_grace_ds=df_grace_ds.rename(columns={'TWS Change [mm/year]':'dS_GRACE'})
ts_all.append(df_grace_ds)

### Read monthly Qout
for key in ['Qoutlet','Qswout','Qgwout']:
    if BASIN['input_ts'][key] is None:
        Q=ts_all[0]*0
        Q.columns=[key]
        ts_all.append(Q)
    else:        
        Q_m=pd.read_csv(BASIN['input_ts'][key],sep=';',index_col=0,skiprows=0)
        Q_m.index=[datetime.datetime.strptime(y,'%d/%m/%Y %H:%M') for y in Q_m.index]
        Q_m=Q_m.replace(-9999,np.nan)
        Q_y=Q_m.resample('A-{0}'.format(BASIN['end_month'])).sum() #mean()
        Q_y.index=[y.year for y in Q_y.index]
        ##
        df_Q_y=Q_y.where(Q_y.days>=365) ##remove years with missing data
        df_Q_y=df_Q_y['km3/month']*1000000 ##convert to TCM=km2*mm
        df_Q_y=df_Q_y.dropna()
        df_Q_y.name=key
        Q=df_Q_y.to_frame()
        ts_all.append(Q)

# 5.3 Generate output table

In [ ]:
for i in range(len(ts_all)):
    if i ==0:
        df=ts_all[i]        
    else:
        df=pd.merge(df,ts_all[i],left_index=True,right_index=True,how='inner')
        
df['dS_WB']=df['Precipitation']-df['Actual Evapotranspiration']-df['Qoutlet']-df['Qswout']-df['Qgwout']
df['dS_Error']=df['dS_GRACE']-df['dS_WB']

OUT_CSV=os.path.join(BASIN['Dir'],'df_all.csv')
df.to_csv(OUT_CSV,sep=';')
BASIN['main_data']['df_all']=OUT_CSV
pickle_out(BASIN)

In [ ]:
#Get sheet1 csv template
WORKING_DIR=WA.__path__[0]
csv_template=os.path.join(WORKING_DIR,'csv','Sample_sheet1.csv')

convert_unit=1000000 #from TCM to BCM
# convert_unit=1000 #from TCM to MCM

df=pd.read_csv(BASIN['main_data']['df_all'],sep=';',index_col=0)
df

# 5.4 Generate yearly csv files

In [ ]:
df=df/convert_unit
csv_folder=os.path.join(BASIN['Dir'],'data','csv')
if not os.path.exists(csv_folder):
    os.makedirs(csv_folder)

df_template=pd.read_csv(csv_template,sep=";")
csv_fhs=[]
for year in df.index:
    df_year=df_template.copy()
    df_year['VALUE']=0.0
    df_year.loc[0,'VALUE']=df.loc[year,'Precipitation']
#    if df.loc[year,'dS']>0:
#        df_year.loc[11,'VALUE']=df.loc[year,'dS_WB']
#    else:
#        df_year.loc[10,'VALUE']=abs(df.loc[year,'dS_WB'])
    df_year.loc[10,'VALUE']=-df.loc[year,'dS_WB']
    df_year.loc[12,'VALUE']=df.loc[year,'Rainfall_ET_M-Protected Landuse']
    df_year.loc[13,'VALUE']=df.loc[year,'Rainfall_ET_M-Utilized Landuse']
    df_year.loc[14,'VALUE']=df.loc[year,'Rainfall_ET_M-Modified Landuse']
    df_year.loc[15,'VALUE']=df.loc[year,'Rainfall_ET_M-Managed Water Use']
    df_year.loc[16,'VALUE']=df.loc[year,'Incremental_ET_M-Protected Landuse']
    df_year.loc[17,'VALUE']=df.loc[year,'Incremental_ET_M-Utilized Landuse']
    df_year.loc[18,'VALUE']=df.loc[year,'Incremental_ET_M-Modified Landuse']
    df_year.loc[19,'VALUE']=df.loc[year,'Incremental_ET_M-Managed Water Use']
    df_year.loc[20,'VALUE']=df.loc[year,'Incremental_ET_M-Managed Water Use']
    df_year.loc[21,'VALUE']=df.loc[year,'Incremental_ET_M']-df.loc[year,'Incremental_ET_M-Managed Water Use']
    df_year.loc[26,'VALUE']=df.loc[year,'Qswout']
    df_year.loc[22,'VALUE']=df.loc[year,'Qoutlet']
    outcsv=os.path.join(csv_folder,'Sheet1_{0}.csv'.format(int(year)))
    df_year.to_csv(outcsv,sep=";",index=False)
    csv_fhs.append(outcsv)

# 5.5 Compute mean csv file (from all the years)

In [ ]:
df_mean = df.mean()  
df_mean_csv=df_template.copy()
df_mean_csv['VALUE']=0.0
df_mean_csv.loc[0,'VALUE']=df_mean.loc['Precipitation']
df_mean_csv.loc[10,'VALUE']=-df_mean.loc['dS_WB']
df_mean_csv.loc[12,'VALUE']=df_mean.loc['Rainfall_ET_M-Protected Landuse']
df_mean_csv.loc[13,'VALUE']=df_mean.loc['Rainfall_ET_M-Utilized Landuse']
df_mean_csv.loc[14,'VALUE']=df_mean.loc['Rainfall_ET_M-Modified Landuse']
df_mean_csv.loc[15,'VALUE']=df_mean.loc['Rainfall_ET_M-Managed Water Use']
df_mean_csv.loc[16,'VALUE']=df_mean.loc['Incremental_ET_M-Protected Landuse']
df_mean_csv.loc[17,'VALUE']=df_mean.loc['Incremental_ET_M-Utilized Landuse']
df_mean_csv.loc[18,'VALUE']=df_mean.loc['Incremental_ET_M-Modified Landuse']
df_mean_csv.loc[19,'VALUE']=df_mean.loc['Incremental_ET_M-Managed Water Use']
df_mean_csv.loc[20,'VALUE']=df_mean.loc['Incremental_ET_M-Managed Water Use']
df_mean_csv.loc[21,'VALUE']=df_mean.loc['Incremental_ET_M']-df.loc[year,'Incremental_ET_M-Managed Water Use']
df_mean_csv.loc[26,'VALUE']=df_mean.loc['Qswout']
df_mean_csv.loc[22,'VALUE']=df_mean.loc['Qoutlet']
outcsv=os.path.join(csv_folder,'Sheet_{}-{}.csv'.format(min(df.index),max(df.index)))
df_mean_csv.to_csv(outcsv,sep=";",index=False)